In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import seaborn as sns

# Exponential Mixture Model

En esta notebook se realizará un modelo similar al Guassian Mixture Model (de aquí en más GMM) pero utilizando en vez de una distribución gaussiana, una exponencial. 


In [7]:
class ExponentialMixtureModel():
    def __init__(self, k: int, max_iter: int = 100, tol: float = 1e-6):
        #Cantidad de Clusters
        self.k = k
        #Maxima cantidad de iteraciones del algirtmo EM
        self.max_iter = max_iter
        #Tolerancia para comprar log-likelihood
        self.tol = tol
        #Pesos /pi's
        self.weights = None
        #Parametro lambda de la exponencial
        self.lambdas = None

    def _prepare_data(self, X: np.ndarray | pd.DataFrame) -> np.ndarray:
        '''
        Prepara los datos de entrada para el algoritmo.

        Args:
        X (array-like): Datos de entrada.
        '''
        if not isinstance(X, (np.ndarray, pd.DataFrame)):
            raise ValueError("X debe ser numpy.ndarray o un pandas.DataFrame.")

        if isinstance(X, pd.DataFrame):
            return X.values
        elif X.ndim == 1:
            return X.reshape(-1, 1)
        return X

    def initialize_parameters(self, X: np.ndarray | pd.DataFrame) -> None:
        '''
        Inicializa los parámetros del modelo.

        Args:
        X (array-like): Datos de entrada.
        '''
        # Preparar los datos
        X = self._prepare_data(X)
        n_samples, n_features = X.shape
        
        # Se asignan los pesos uniformemente
        self.weights = np.full(self.k, 1 / self.k)
        
        # Se inicializan los cuantiles entre valores de 0.2 y 0.8 para cada clase
        quantiles = np.quantile(X, q=np.linspace(0.2, 0.8, self.k), axis=0)
        self.lambdas = 1 / (quantiles + 1e-6)  # 1/ quantiles para "estimar un posible lambda mejor que lo uniforme"
        
    def e_step(self, X: np.ndarray | pd.DataFrame) -> None:
        '''
        Computa el paso E del algoritmo EM.

        Args:
        X (array-like): Datos de entrada.
        '''

        # En este paso se calcula la probabilidad de pertenecer al cluster j dado las observaciones, ponderándolas 
        # por el peso de cada cluster inicializado anteriormente. Es una regla de bayes ponderada con la exponencial como densidad.
        X = self._prepare_data(X)
        n_samples, n_features = X.shape


        self.e_return = np.zeros((n_samples, self.k))
        for k in range(self.k):
            # Calcular la probabilidad de pertenencia de cada observacion a cada cluster
            probs = self.weights[k] * np.prod(self.lambdas[k] * np.exp(-self.lambdas[k] * X), axis=1)

            print(f'Probs for cluster {k}', probs)
            
            # Evitar ceros en las probabilidades
            probs = np.where(probs < 1e-10, 1e-10, probs)
            self.e_return[:, k] = probs
        # Normalizar las probabilidades para que sumen 1 por observación
        self.e_return = self.e_return / self.e_return.sum(axis=1, keepdims=True)
        if np.any(np.isnan(self.e_return)):
            raise ValueError("Advertencia: NaNs encontrados en la matriz de probabilidad después del paso E.")
        

    def m_step(self, X):
        X = self._prepare_data(X)
        n_samples, n_features = X.shape
        old_lambdas = self.lambdas.copy()
        
        for k in range(self.k):
            resp_sum = np.sum(self.e_return[:, k])
            self.weights[k] = resp_sum / n_samples
            
            for j in range(n_features):
                weighted_sum = np.sum(self.e_return[:, k] * X[:, j])
                if weighted_sum < 1e-10:
                    self.lambdas[k, j] = old_lambdas[k, j]  # Mantener valor anterior
                else:
                    # Limitar el cambio en lambda para evitar convergencia rápida a un solo cluster
                    self.lambdas[k, j] = resp_sum / weighted_sum

    def fit(self, X):
        X = self._prepare_data(X)
        self.initialize_parameters(X)
        log_likelihood_old = -np.inf
        
        for i in range(self.max_iter):
            self.e_step(X)
            self.m_step(X)
            
            # Calcular log-likelihood
            #log_probs = np.sum(np.log(np.sum(self.e_return, axis=1) + 1e-300))
            log_probs=0
            for i in range(X.shape[0]):  # Iteramos sobre cada observación
                total_density = 0
                for k in range(self.k):  # Iteramos sobre cada componente
                    # Calculamos la densidad para el componente k, ponderada por su peso
                    density = self.weights[k] * np.prod(self.lambdas[k] * np.exp(-self.lambdas[k] * X[i]))
                    total_density += density
                # Sumamos el logaritmo de la densidad total de la observación i al log-likelihood
                log_probs += np.log(total_density + 1e-300)  # Evitar log(0)

            print("log_probs",log_probs)
            print("np.sum(self.e_return, axis=1):",np.sum(self.e_return, axis=1))

            # Verificar si los clusters están balanceados
            cluster_weights = np.mean(self.e_return, axis=0)
            max_weight_ratio = np.max(cluster_weights) / np.min(cluster_weights)
            
            print(f"Log-likelihood: {log_probs}")
            print(f"Ratio max/min de pesos de clusters: {max_weight_ratio}")
            
            if i > 0 and np.abs(log_probs - log_likelihood_old) < self.tol:
                if max_weight_ratio > 100:  # Si los clusters están muy desbalanceados
                    print("Reiniciando debido a clusters desbalanceados...")
                    self.initialize_parameters(X)
                    log_likelihood_old = -np.inf
                    continue
                    
                print(f"Convergencia alcanzada en la iteración {i+1}")
                break
                
            log_likelihood_old = log_probs

    def predict_proba(self, X):
        X = self._prepare_data(X)
        self.e_step(X)
        return self.e_return

    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)

# Ejemplo de uso

In [57]:
np.random.seed(0)
# X = np.concatenate([
#     np.random.exponential(scale=1/2, size=100),  
#     np.random.exponential(scale=1/5, size=100) 
# ])

X = pd.DataFrame({
    'X1': np.random.exponential(scale=1/2, size=100),  
    'X2': np.random.exponential(scale=1/5, size=100)})

# Crear y ajustar el modelo
emm = ExponentialMixtureModel(k=2, max_iter=1000)
emm.fit(X)

# Predicciones
print("Pesos de mezcla:\n", emm.weights)
print("Tasas (lambda):\n", emm.lambdas)

clusters = emm.predict(X)
print("\nAsignación de clusters:", clusters)
print("Matriz de probabilidad:",emm.predict_proba(X))

Pesos iniciales:
 [0.5 0.5]
Parámetros iniciales (lambdas):
 [[3.80616294 7.2670914 ]
 [4.39976655 2.6164331 ]]
Iteración 0, log-likelihood: 9.999997274190034e-09
Convergencia alcanzada en la iteración 1.
Pesos de mezcla:
 [0.54749549 0.45250451]
Tasas (lambda):
 [[1.83040555 6.34048387]
 [2.82439975 3.65110362]]

Asignación de clusters: [0 0 0 1 0 0 0 0 0 1 0 1 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0
 0 0 0 1 0 0 1 1 0 0 1 1 1 0 1 0 1 0 1 1 0 0 0 1 0 0 1 1 1 0 0 0 0 0 0 0 0
 0 0 0 1 1 0 0 0 1 0 1 1 0 1 0 0 0 0 0 1 1 0 0 0 0 0]
Matriz de probabilidad: [[0.52374571 0.47625429]
 [0.68214273 0.31785727]
 [0.51321092 0.48678908]
 [0.25699224 0.74300776]
 [0.60557501 0.39442499]
 [0.58976087 0.41023913]
 [0.52809547 0.47190453]
 [0.72252746 0.27747254]
 [0.8606207  0.1393793 ]
 [0.25111459 0.74888541]
 [0.6834565  0.3165435 ]
 [0.41948483 0.58051517]
 [0.51980567 0.48019433]
 [0.80379236 0.19620764]
 [0.36382919 0.63617081]
 [0.52058662 0.47941338]
 [0.30437897 0.69562103]
 [0.674

In [19]:
X = np.concatenate([
    np.random.exponential(scale=1/2, size=100),  
    np.random.exponential(scale=1/5, size=100)
])

emm = ExponentialMixtureModel(k=2, max_iter=10)
emm.initialize_parameters(X)
# emm.e_step(X)

In [23]:
emm.weights

array([0.5, 0.5])

In [28]:
X = X.reshape(-1, 1)
n_samples, n_features = X.shape

e_return = np.zeros((n_samples, 2))
for k in range(2):
    print('Weight:', emm.weights[k])
    print('Lambda:', emm.lambdas[k])
    print('Shape Exponential:', np.exp(-emm.lambdas[k] * X).shape)
    print('Shape Product:', np.prod(emm.lambdas[k] * np.exp(-emm.lambdas[k] * X), axis=1))
    probs = emm.weights[k] * np.prod(emm.lambdas[k] * np.exp(-emm.lambdas[k] * X), axis=1)

Weight: 0.5
Lambda: [15.5674366]
Shape Exponential: (200, 1)
Shape Product: [6.58391769e-06 1.02564251e-04 2.19355821e+00 2.20534289e-01
 1.42526997e-01 2.10854696e-05 4.77976875e-02 1.95936370e+00
 5.63152013e-08 1.35207663e-03 6.09620066e-04 1.22979436e+00
 1.39316410e+01 4.81501805e-04 4.09292431e+00 2.13882638e-07
 2.27344949e-02 2.64476307e-03 2.16436614e-01 8.45562150e-02
 2.76258704e-05 3.83936666e+00 1.44134926e+01 3.78608441e-04
 5.72817754e-04 6.05119697e+00 3.50609114e-01 3.25002385e-05
 1.33360684e+01 6.46602239e+00 1.44043431e+00 5.40186086e+00
 9.53042577e-06 2.54996789e-01 4.29441283e-05 3.52048539e-10
 7.48343390e-08 3.73632768e-01 6.89636737e-03 4.10621020e-02
 1.38711813e-06 3.64513015e+00 2.03034213e-04 1.34739224e+01
 1.10613046e-01 2.88346613e-02 7.06886206e-03 1.33178725e-03
 2.03401490e-04 2.82474850e-07 7.91269159e-01 1.27611551e+01
 1.07845666e+00 1.03112274e+01 6.92804297e-06 6.99510679e-02
 1.16478372e-01 1.03414956e+01 4.95798132e-04 7.90478992e-03
 1.137255